#### MIRA Course -- Non-rigid registration 
Todo's:
1.	Try to localize the points in the given images. ok
2.	Calculate the initial error without registration. ok
3.	Calculate non rigid registration on intensity images and on the points.
4.	Calculate the non-rigid registration errors. 

In [1]:
# Import Libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [2]:
# Read nifti files
def read_nifti_file(filepath):
    # Read the image using SimpleITK
    itkimage = sitk.ReadImage(filepath)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    return ct_scan, origin, spacing

In [20]:
image = sitk.ReadImage("../data/nifti/copd1_iBHCT.nii.gz")
print(image.GetSize())
print(image.GetOrigin())
print(image.GetSpacing())
print(image.GetDirection())
print(image.GetNumberOfComponentsPerPixel())

(512, 512, 121)
(0.0, 0.0, 0.0)
(0.6200000047683716, 0.6200000047683716, 2.5)
(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
1


In [23]:
# Read landmarks 
def get_landmark(p, i):
    """get the landmarks of the patient

    Args:
        p (str): inhale or exhale (i or e)
        i (int): patient number

    Returns:
        _type_: pandas dataframe
    """
    image = sitk.ReadImage(f"../data/nifti/copd{i}_{p}BHCT.nii.gz")
    file= f"../data/keypoints/copd{i}_300_{p}BH_xyz_r1.txt"
    x,y,z = image.GetSpacing()
    if p == "i":
        landmark = pd.read_csv(file, sep="\t", header=None)
    else:
        landmark = pd.read_csv(file, sep="\t ", header=None)
    
    # Multiply spacing to get distance in mm
    landmark = landmark * np.array([x,y,z])

    return np.array(landmark)

get_landmark(p="i", i=1)


array([[116.5600009 , 161.20000124,  10.        ],
       [110.98000085, 174.22000134,  17.5       ],
       [115.94000089, 156.86000121,  20.        ],
       [ 99.82000077, 195.3000015 ,  22.5       ],
       [107.26000082, 149.42000115,  22.5       ],
       [196.54000151, 150.04000115,  25.        ],
       [192.20000148, 197.78000152,  25.        ],
       [174.84000134, 169.2600013 ,  27.5       ],
       [122.76000094, 145.08000112,  27.5       ],
       [133.30000103, 158.72000122,  30.        ],
       [117.80000091, 192.82000148,  30.        ],
       [187.86000144, 197.78000152,  30.        ],
       [175.46000135, 143.2200011 ,  32.5       ],
       [112.22000086, 145.70000112,  32.5       ],
       [ 96.72000074, 145.08000112,  32.5       ],
       [116.5600009 , 182.90000141,  32.5       ],
       [222.58000171, 172.36000133,  35.        ],
       [212.04000163, 213.90000165,  37.5       ],
       [228.78000176, 176.70000136,  37.5       ],
       [114.70000088, 141.36000

In [24]:
# Calculate TRE
def calculate_tre(fixed_points, moving_points):
    tre = np.sqrt(np.sum((fixed_points - moving_points) ** 2, axis=1))
    return tre

for k in range(1,5):
    fixed_points = get_landmark(p="i", i=k)
    moving_points = get_landmark(p="e", i=k)
    mean = np.mean(calculate_tre(fixed_points, moving_points))
    std = np.std(calculate_tre(fixed_points, moving_points))
    print(f"Mean: {mean}, Std: {std}")

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Mean: 26.14727660137311, Std: 11.337752042730125
Mean: 21.640839694139274, Std: 6.415267547626306
Mean: 12.62588335216309, Std: 6.381867428751379
Mean: 29.583559738904107, Std: 12.92417092574431
